# Analyse exploratoire, traitements, modélisation des données

## 0 - Chargement des librairies et du dataset

In [3]:
import pandas as pd
from skimpy import skim 

In [2]:
df = pd.read_csv(r'https://raw.githubusercontent.com/MaskiVal/DataSets/main/cancer_breast.csv')

## 1 - Exploration des données

#### 1.1.1 - Nombre de données, visualisation des colonnes du dataset

In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
id,569.0,3.037183e+07,1.250206e+08,8670.000000,869218.000000,906024.000000,8.813129e+06,9.113205e+08
radius_mean,569.0,1.412729e+01,3.524049e+00,6.981000,11.700000,13.370000,1.578000e+01,2.811000e+01
texture_mean,569.0,1.928965e+01,4.301036e+00,9.710000,16.170000,18.840000,2.180000e+01,3.928000e+01
perimeter_mean,569.0,9.196903e+01,2.429898e+01,43.790000,75.170000,86.240000,1.041000e+02,1.885000e+02
area_mean,569.0,6.548891e+02,3.519141e+02,143.500000,420.300000,551.100000,7.827000e+02,2.501000e+03
smoothness_mean,569.0,9.636028e-02,1.406413e-02,0.052630,0.086370,0.095870,1.053000e-01,1.634000e-01
compactness_mean,569.0,1.043410e-01,5.281276e-02,0.019380,0.064920,0.092630,1.304000e-01,3.454000e-01
concavity_mean,569.0,8.879932e-02,7.971981e-02,0.000000,0.029560,0.061540,1.307000e-01,4.268000e-01
concave points_mean,569.0,4.891915e-02,3.880284e-02,0.000000,0.020310,0.033500,7.400000e-02,2.012000e-01
symmetry_mean,569.0,1.811619e-01,2.741428e-02,0.106000,0.161900,0.179200,1.957000e-01,3.040000e-01


In [6]:
skim(df)

c:\Users\dimle\Documents\clone_repo\semiology_AI\.conda\Lib\site-packages\numpy\lib\histograms.py:885: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges


╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 569    │ │ float64     │ 31    │                                                          │
│ │ Number of columns │ 33     │ │ int32       │ 1     │                                                          │
│ └───────────────────┴────────┘ │ string      │ 1     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━┳━━━━━┳━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┓  │
│ ┃ column_n ┃     ┃      ┃          ┃          ┃          ┃          ┃          ┃          ┃         ┃        ┃  │
│ ┃ ame      ┃ NA  ┃ NA % ┃ mean     ┃ sd       ┃ p0       ┃ p25      ┃ p50      ┃ p75      ┃ p100    ┃ hist   ┃  │
│ ┡━━━━━━━━━━╇━━━━━╇━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━┩  │
│ │ id       │   0 │    0 │ 30370000 │ 12500000 │     8670 │   869200 │   906000 │  8813000 │ 9113000 │   ▇    │  │
│ │          │     │      │          │        0 │          │          │          │          │      00 │        │  │
│ │ radius_m │   0 │    0 │    14.13 │    3.524 │    6.981 │     11.7 │    13.37 │    15.78 │   28.11 │  ▂▇▃▂  │  │
│ │ ean      │     │      │          │          │          │          │          │          │         │        │  │
│ │ texture_ │   0 │    0 │    19.29 │    4.301 │     9.71 │    16.17 │    18.84 │     21.8 │   39.28 │  ▂▇▆▂  │  │
│ │ mean     │     │      │          │          │          │          │          │          │         │        │  │
│ │ perimete │   0 │    0 │    91.97 │     24.3 │    43.79 │    75.17 │    86.24 │    104.1 │   188.5 │  ▂▇▃▂  │  │
│ │ r_mean   │     │      │          │          │          │          │          │          │         │        │  │
│ │ area_mea │   0 │    0 │    654.9 │    351.9 │    143.5 │    420.3 │    551.1 │    782.7 │    2501 │  ▇▆▂▁  │  │
│ │ n        │     │      │          │          │          │          │          │          │         │        │  │
│ │ smoothne │   0 │    0 │  0.09636 │  0.01406 │  0.05263 │  0.08637 │  0.09587 │   0.1053 │  0.1634 │   ▅▇▃  │  │
│ │ ss_mean  │     │      │          │          │          │          │          │          │         │        │  │
│ │ compactn │   0 │    0 │   0.1043 │  0.05281 │  0.01938 │  0.06492 │  0.09263 │   0.1304 │  0.3454 │  ▇▇▃▁  │  │
│ │ ess_mean │     │      │          │          │          │          │          │          │         │        │  │
│ │ concavit │   0 │    0 │   0.0888 │  0.07972 │        0 │  0.02956 │  0.06154 │   0.1307 │  0.4268 │  ▇▃▂▁  │  │
│ │ y_mean   │     │      │          │          │          │          │          │          │         │        │  │
│ │ concave  │   0 │    0 │  0.04892 │   0.0388 │        0 │  0.02031 │   0.0335 │    0.074 │  0.2012 │  ▇▃▃▁  │  │
│ │ points_m │     │      │          │          │          │          │          │          │         │        │  │
│ │ ean      │     │      │          │          │          │          │          │          │         │        │  │
│ │ symmetry │   0 │    0 │   0.1812 │  0.02741 │    0.106 │   0.1619 │   0.1792 │   0.1957 │   0.304 │ ▁▇▇▂▁  │  │
│ │ _mean    │     │      │          │          │       

#### 1.1.2 - Observation des types de données

In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
id,569.0,3.037183e+07,1.250206e+08,8670.000000,869218.000000,906024.000000,8.813129e+06,9.113205e+08
radius_mean,569.0,1.412729e+01,3.524049e+00,6.981000,11.700000,13.370000,1.578000e+01,2.811000e+01
texture_mean,569.0,1.928965e+01,4.301036e+00,9.710000,16.170000,18.840000,2.180000e+01,3.928000e+01
perimeter_mean,569.0,9.196903e+01,2.429898e+01,43.790000,75.170000,86.240000,1.041000e+02,1.885000e+02
area_mean,569.0,6.548891e+02,3.519141e+02,143.500000,420.300000,551.100000,7.827000e+02,2.501000e+03
smoothness_mean,569.0,9.636028e-02,1.406413e-02,0.052630,0.086370,0.095870,1.053000e-01,1.634000e-01
compactness_mean,569.0,1.043410e-01,5.281276e-02,0.019380,0.064920,0.092630,1.304000e-01,3.454000e-01
concavity_mean,569.0,8.879932e-02,7.971981e-02,0.000000,0.029560,0.061540,1.307000e-01,4.268000e-01
concave points_mean,569.0,4.891915e-02,3.880284e-02,0.000000,0.020310,0.033500,7.400000e-02,2.012000e-01
symmetry_mean,569.0,1.811619e-01,2.741428e-02,0.106000,0.161900,0.179200,1.957000e-01,3.040000e-01


#### 1.1.3 - Observation des valeurs Null

### 1.2 - Observation des variables numériques

#### 1.2.1 - Analyse de la distribution

#### 1.2.2 - Analyse statique descriptive de base

### 1.3 - Observation des variables catégoritielles

#### 1.3.1 - Analyse du nombre de valeurs unique

#### 1.3.2 - Analyse des  statistique de base

## 2 - Nettoyage des données

### 2.1 - Traitement des valeurs manquantes

### 2.2 - Traitement des doublons

### 2.3 - Traitement des valeurs abérantes

## 3 - Visualisation des données nettoyées

## 4 - Modélisation des données